# Read the full dataset

In [1]:
import rasterio
import numpy as np

# https://data.worldpop.org/GIS/Population/Global_2000_2020/2020/0_Mosaicked/ppp_2020_1km_Aggregated.tif
population_data = rasterio.open('data/ppp_2020_1km_Aggregated.tif')
#show(population_data)

In [2]:
band1 = population_data.read(1)

In [3]:
# Set NA values
band1[band1 == -3.4028234663852886e+38] = np.nan
np.sum(band1, where=~np.isnan(band1))

7966755000.0

In [48]:
def get_population(tif_file, na_value = -3.4028234663852886e+38):
    population_data = rasterio.open(tif_file)
    band1 = population_data.read(1)
    band1[band1 == na_value] = np.nan
    return np.sum(band1, where=~np.isnan(band1))
total_population = get_population('data/ppp_2020_1km_Aggregated.tif')
total_population

7966755000.0

# Get population in a box around NL

- This solution takes far too much RAM: https://gis.stackexchange.com/questions/388047/get-coordinates-of-all-pixels-in-a-raster-with-rasterio
- https://stackoverflow.com/questions/47404898/find-indices-of-raster-cells-that-intersect-with-a-polygon

Beacuse of slowness we opt to use a library to do the work for us: GDAL. 

In [5]:
# https://gis.stackexchange.com/questions/294206/create-a-polygon-from-coordinates-in-geopandas-with-python
# Bbox from https://gist.github.com/graydon/11198540: (3.31497114423, 50.803721015, 7.09205325687, 53.5104033474)
# The points are from top left, clockwise

import geopandas as gpd
from shapely.geometry import Polygon

lat_point_list = [53.5104033474, 53.5104033474, 50.803721015, 50.803721015]
lon_point_list = [3.31497114423, 7.09205325687, 7.09205325687, 3.31497114423]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
bbox_nl = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])  

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [106]:
def plot_folium(shape, center, zoom):
    f = folium.Figure(width=300, height=300)
    m = folium.Map(center, zoom_start=zoom, tiles='cartodbpositron').add_to(f)
    folium.GeoJson(shape).add_to(m)
    folium.LatLngPopup().add_to(m)
    return m
plot_folium(bbox_nl, [51.854457, 4.377184], 5)

In [7]:
# Dump the shape file
bbox_nl.to_file(filename='tmp/bbox_nl.shp', driver="ESRI Shapefile")

Note that getting GDAL working is quite a bit of a hassle. If you use my Docker image all the tooling is already installed. 

In [8]:
# https://gis.stackexchange.com/questions/45053/gdalwarp-cutline-along-with-shapefile
# gdalwarp -cutline INPUT.shp -crop_to_cutline -dstalpha INPUT.tif OUTPUT.tif
from osgeo import gdal

# Note input and output are reversed for some reason
ds = gdal.Warp('tmp/bbox_nl.tif', 'data/ppp_2020_1km_Aggregated.tif', cutlineDSName='tmp/bbox_nl.shp', cropToCutline=True)
del ds

In [9]:
get_population('tmp/bbox_nl.tif') / 1e6

31.186402

In [29]:
from osgeo import gdal

def get_population_in_shape(shape_obj):
    shape_obj.to_file(filename='tmp/bbox_nl.shp', driver="ESRI Shapefile")
    ds = gdal.Warp('tmp/bbox_nl.tif', 'data/ppp_2020_1km_Aggregated.tif', cutlineDSName='tmp/bbox_nl.shp', cropToCutline=True)
    del ds
    return get_population('tmp/bbox_nl.tif')
get_population_in_shape(bbox_nl)

31186402.0

Sounds about right. Netherlands has a population of around 17m, and some populuous areas of Belgium and Germany are also included. 

# Getting population inside a circle

In [125]:
from functools import partial

import pyproj
from shapely import geometry
from shapely.geometry import Point
from shapely.ops import transform

def get_circle_with_radius(lon, lat, radius, resolution=16):
    # https://gis.stackexchange.com/questions/121256/creating-a-circle-with-radius-in-metres
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={lat} +lon_0={lon}".format(
        lat, lon
    )
    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
        pyproj.Proj(local_azimuthal_projection),
    )
    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
    )

    center = Point(float(lon), float(lat))
    point_transformed = transform(wgs84_to_aeqd, center)
    buffer = point_transformed.buffer(radius, resolution)
    # Get the polygon with lat lon coordinates
    circle_poly = transform(aeqd_to_wgs84, buffer)

    return gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[circle_poly])

In [126]:
greenland_circle = get_circle_with_radius(-45, 63, 2200e3, resolution=64)
plot_folium(greenland_circle, [63, -45], 1)

In [127]:
get_population_in_shape(greenland_circle) / 1e6

1.81848775

Not a lot of people live here, and the circle is quite oval shaped :). 

# Get population inside the RLL video
https://www.youtube.com/watch?v=mcqq8eAufXk&t=643s

In [131]:
# 3418e3
mong_khet_circle = get_circle_with_radius(99.38, 21.72, 3412e3, resolution=64)
mk_circle_population = get_population_in_shape(mong_khet_circle) 
print(mk_circle_population / 1e6)
mk_circle_population / total_population

3983.971328


0.50007457

In [129]:
plot_folium(mong_khet_circle, [21.72, 99.38], 2)

This circle is slightly different in terms of radius, but that ofcourse also depends on the input data used. 